In [3]:
from pathlib import Path
from rich import print  # pretty printing

import os
from semantic_router.encoders import OpenAIEncoder

import pandas as pd
import re

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") 
encoder = OpenAIEncoder(name="text-embedding-3-small")

In [25]:
def read_txt_as_text(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

file_path = 'extracted_data.txt'  # Replace with your actual PDF file path
current_dir = Path().resolve()
file_path = current_dir.parent / "data" / file_path

    
text = read_txt_as_text(file_path)
short_text = text[:5_000]

In [27]:
class SplitSections(dspy.Signature):
    headings: list[dict[str, str]] = dspy.OutputField(desc='each heading and its content')
    


In [28]:
import nltk
from sentence_transformers import SentenceTransformer, util


In [29]:
model = SentenceTransformer("all-MiniLM-L6-v2")
sentences = nltk.sent_tokenize(short_text)

print(sentences)

2025-08-11 17:43:43 - sentence_transformers.SentenceTransformer - INFO - SentenceTransformer.py:219 - __init__() - Use pytorch device_name: cuda:0
2025-08-11 17:43:43 - sentence_transformers.SentenceTransformer - INFO - SentenceTransformer.py:227 - __init__() - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


[
    'Title: None\nDOI: 10.3390/su2113436\nFullTextURL: http://creativecommons.org/licenses/by/3.0/).',
    'FullTextContent:\nSustainability 2010, 2, 3436-3448; doi:10.3390/su2113436 OPEN ACCESS sustainability ISSN 
2071-1050 www.mdpi.com/journal/sustainability Review What is Sustainability?',
    'Tom Kuhlman 1,* and John Farrington 2 1 Agricultural Economics Research Institute, Wageningen University, 
P.O.',
    'Box 29703, 2502LS The Hague, The Netherlands 2 Institute for Rural Research, Geography and Environment, 
University of Aberdeen, Elphinstone Road, Aberdeen AB24 3UF, Scotland, UK; E-Mail: j.farrington@abdn.ac.uk * Author
to whom correspondence should be addressed; E-Mail: tom.kuhlman@wur.nl; Tel.',
    ': +31-70-3358-232; Fax: +31-70-3615-624.',
    'Received: 17 September 2010; in revised form: 15 October 2010 / Accepted: 19 October 2010 / Published: 1 
November 2010 Abstract: Sustainability as a policy concept has its origin in the Brundtland Report of 1987.',
    'That document was concerned with the tension between the aspirations of mankind towards a better life on the 
one hand and the limitations imposed by nature on the other hand.',
    'In the course of time, the concept has been re-interpreted as encompassing three dimensions, namely social, 
economic and environmental.',
    'The paper argues that this change in meaning (a) obscures the real contradiction between the aims of welfare 
for all and environmental conservation; (b) risks diminishing the importance of the environmental dimension; and 
(c) separates social from economic aspects, which in reality are one and the same.',
    'It is proposed instead to return to the original meaning, where sustainability is concerned with the 
well-being of future generations and in particular with irreplaceable natural resources as opposed to the 
gratification of present needs which we call well-being.',
    'A balance needs to be found between those two, but not by pretending they are three sides of the same coin.',
    'Although we use up natural resources at the expense of future generations, we also generate capital (including
knowledge) which raises future well-being.',
    'A major question is to what extent the one compensates for the other.',
    'This debate centres around the problem of substitutability, which has been cast into a distinction between  
weak  and  strong  sustainability.',
    'It is argued that these two do not need to be in opposition but complement one another.',
    'Sustainability 2010, 2 3437 Keywords: sustainability; well-being; welfare \n\n1.',
    'Introduction \nThe term sustainability has become popular in policy-oriented research as an expression of what
public policies ought to achieve.',
    'The principal inspiration came from the Brundtland Report of 1987 [1].',
    'Since then the concept has shifted in meaning.',
    'This paper argues that the shift is unfortunate in that it obscures the real contradiction which exists 
between long-term sustainability and short-term welfare.',
    'Moreover, the distinction between three  pillars of sustainability is conceptually fuzzy.',
    'We propose a definition that reverts to the original sense in which the concept was intended.',
    'However, this paper does not pretend to offer a comprehensive view of the problem of sustainability.',
    'It presents a critical view of how the term is used in policy debate and in impact assessment the set of 
methods used in applied research to appraise policies and projects.',
    '2.',
    'History of the Concept The concept of sustainability was originally coined in forestry, where it means never 
harvesting more than what the forest yields in new growth [2].',
    'The word Nachhaltigkeit (the German term for sustainability) was first used with this meaning in 1713 [3].',
    'The concern with preserving natural resources for the future is perennial, of course: undoubtedly our 
Palaeolithic ancestors worried about their prey becoming 

In [30]:
len(sentences)

34

In [42]:


embeddings = model.encode(sentences, convert_to_tensor=True)
splits = [0]

for i in range(len(sentences) - 1):
    sim = util.cos_sim(embeddings[i], embeddings[i+1]).item()
    if sim < 0.50:  # threshold
        splits.append(i+1)
        
len(splits)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

31

In [ ]:

splits.append(len(sentences))

chunks = [" ".join(sentences[s:e]) for s, e in zip(splits, splits[1:])]
print(len(chunks))

34

In [33]:
print(chunks)

[
    'Title: None\nDOI: 10.3390/su2113436\nFullTextURL: http://creativecommons.org/licenses/by/3.0/).',
    'FullTextContent:\nSustainability 2010, 2, 3436-3448; doi:10.3390/su2113436 OPEN ACCESS sustainability ISSN 
2071-1050 www.mdpi.com/journal/sustainability Review What is Sustainability?',
    'Tom Kuhlman 1,* and John Farrington 2 1 Agricultural Economics Research Institute, Wageningen University, 
P.O.',
    'Box 29703, 2502LS The Hague, The Netherlands 2 Institute for Rural Research, Geography and Environment, 
University of Aberdeen, Elphinstone Road, Aberdeen AB24 3UF, Scotland, UK; E-Mail: j.farrington@abdn.ac.uk * Author
to whom correspondence should be addressed; E-Mail: tom.kuhlman@wur.nl; Tel.',
    ': +31-70-3358-232; Fax: +31-70-3615-624.',
    'Received: 17 September 2010; in revised form: 15 October 2010 / Accepted: 19 October 2010 / Published: 1 
November 2010 Abstract: Sustainability as a policy concept has its origin in the Brundtland Report of 1987.',
    'That document was concerned with the tension between the aspirations of mankind towards a better life on the 
one hand and the limitations imposed by nature on the other hand.',
    'In the course of time, the concept has been re-interpreted as encompassing three dimensions, namely social, 
economic and environmental.',
    'The paper argues that this change in meaning (a) obscures the real contradiction between the aims of welfare 
for all and environmental conservation; (b) risks diminishing the importance of the environmental dimension; and 
(c) separates social from economic aspects, which in reality are one and the same.',
    'It is proposed instead to return to the original meaning, where sustainability is concerned with the 
well-being of future generations and in particular with irreplaceable natural resources as opposed to the 
gratification of present needs which we call well-being.',
    'A balance needs to be found between those two, but not by pretending they are three sides of the same coin.',
    'Although we use up natural resources at the expense of future generations, we also generate capital (including
knowledge) which raises future well-being.',
    'A major question is to what extent the one compensates for the other.',
    'This debate centres around the problem of substitutability, which has been cast into a distinction between  
weak  and  strong  sustainability.',
    'It is argued that these two do not need to be in opposition but complement one another.',
    'Sustainability 2010, 2 3437 Keywords: sustainability; well-being; welfare \n\n1.',
    'Introduction \nThe term sustainability has become popular in policy-oriented research as an expression of what
public policies ought to achieve.',
    'The principal inspiration came from the Brundtland Report of 1987 [1].',
    'Since then the concept has shifted in meaning.',
    'This paper argues that the shift is unfortunate in that it obscures the real contradiction which exists 
between long-term sustainability and short-term welfare.',
    'Moreover, the distinction between three  pillars of sustainability is conceptually fuzzy.',
    'We propose a definition that reverts to the original sense in which the concept was intended.',
    'However, this paper does not pretend to offer a comprehensive view of the problem of sustainability.',
    'It presents a critical view of how the term is used in policy debate and in impact assessment the set of 
methods used in applied research to appraise policies and projects.',
    '2.',
    'History of the Concept The concept of sustainability was originally coined in forestry, where it means never 
harvesting more than what the forest yields in new growth [2].',
    'The word Nachhaltigkeit (the German term for sustainability) was first used with this meaning in 1713 [3].',
    'The concern with preserving natural resources for the future is perennial, of course: undoubtedly our 
Palaeolithic ancestors worried about their prey becoming 

In [13]:
print(chunks[2])

Tom Kuhlman 1,* and John Farrington 2 1 Agricultural Economics Research Institute, Wageningen University, P.O.

In [19]:
# --- Second pass: merge chunks if they are still semantically close ---
merged_chunks = []
i = 0
while i < len(chunks):
    if i < len(chunks) - 1:
        # Embed the current and next chunk
        emb1 = model.encode(chunks[i], convert_to_tensor=True)
        emb2 = model.encode(chunks[i+1], convert_to_tensor=True)
        sim = util.cos_sim(emb1, emb2).item()

        if sim >= 0.80:  # merge threshold (slightly higher)
            merged_chunks.append(chunks[i] + " " + chunks[i+1])
            i += 2
            continue
    merged_chunks.append(chunks[i])
    i += 1

print(merged_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[
    'Title: None\nDOI: 10.3390/su2113436\nFullTextURL: http://creativecommons.org/licenses/by/3.0/).',
    'FullTextContent:\nSustainability 2010, 2, 3436-3448; doi:10.3390/su2113436 OPEN ACCESS sustainability ISSN 
2071-1050 www.mdpi.com/journal/sustainability Review What is Sustainability?',
    'Tom Kuhlman 1,* and John Farrington 2 1 Agricultural Economics Research Institute, Wageningen University, 
P.O.',
    'Box 29703, 2502LS The Hague, The Netherlands 2 Institute for Rural Research, Geography and Environment, 
University of Aberdeen, Elphinstone Road, Aberdeen AB24 3UF, Scotland, UK; E-Mail: j.farrington@abdn.ac.uk * Author
to whom correspondence should be addressed; E-Mail: tom.kuhlman@wur.nl; Tel.',
    ': +31-70-3358-232; Fax: +31-70-3615-624.',
    'Received: 17 September 2010; in revised form: 15 October 2010 / Accepted: 19 October 2010 / Published: 1 
November 2010 Abstract: Sustainability as a policy concept has its origin in the Brundtland Report of 1987.',
    'That document was concerned with the tension between the aspirations of mankind towards a better life on the 
one hand and the limitations imposed by nature on the other hand.',
    'In the course of time, the concept has been re-interpreted as encompassing three dimensions, namely social, 
economic and environmental.',
    'The paper argues that this change in meaning (a) obscures the real contradiction between the aims of welfare 
for all and environmental conservation; (b) risks diminishing the importance of the environmental dimension; and 
(c) separates social from economic aspects, which in reality are one and the same.',
    'It is proposed instead to return to the original meaning, where sustainability is concerned with the 
well-being of future generations and in particular with irreplaceable natural resources as opposed to the 
gratification of present needs which we call well-being.',
    'A balance needs to be found between those two, but not by pretending they are three sides of the same coin.',
    'Although we use up natural resources at the expense of future generations, we also generate capital (including
knowledge) which raises future well-being.',
    'A major question is to what extent the one compensates for the other.',
    'This debate centres around the problem of substitutability, which has been cast into a distinction between  
weak  and  strong  sustainability.',
    'It is argued that these two do not need to be in opposition but complement one another.',
    'Sustainability 2010, 2 3437 Keywords: sustainability; well-being; welfare \n\n1.',
    'Introduction \nThe term sustainability has become popular in policy-oriented research as an expression of what
public policies ought to achieve.',
    'The principal inspiration came from the Brundtland Report of 1987 [1].',
    'Since then the concept has shifted in meaning.',
    'This paper argues that the shift is unfortunate in that it obscures the real contradiction which exists 
between long-term sustainability and short-term welfare.',
    'Moreover, the distinction between three  pillars of sustainability is conceptually fuzzy.',
    'We propose a definition that reverts to the original sense in which the concept was intended.',
    'However, this paper does not pretend to offer a comprehensive view of the problem of sustainability.',
    'It presents a critical view of how the term is used in policy debate and in impact assessment the set of 
methods used in applied research to appraise policies and projects.',
    '2.',
    'History of the Concept The concept of sustainability was originally coined in forestry, where it means never 
harvesting more than what the forest yields in new growth [2].',
    'The word Nachhaltigkeit (the German term for sustainability) was first used with this meaning in 1713 [3].',
    'The concern with preserving natural resources for the future is perennial, of course: undoubtedly our 
Palaeolithic ancestors worried about their prey becoming 